In [2]:
import pandas as pd
import os

# Data Reading (ignore if have CSVs)

In [2]:
dir = 'X:\\PhD\\SemEval Task4\\Data'


anno_subtask1_train = pd.read_json('X:\\PhD\\SemEval Task4\\Data\\annotations\\data\\subtask1\\train.json')
anno_subtask1_val = pd.read_json('X:\\PhD\\SemEval Task4\\Data\\annotations\\data\\subtask1\\validation.json')
anno_subtask1_dev = pd.read_json('X:\\PhD\\SemEval Task4\\Data\\annotations\\data\\subtask1\\dev_unlabeled.json')

anno_subtask1_combined = pd.concat([anno_subtask1_train, anno_subtask1_val])
anno_subtask1_combined

,id,text,labels,link
0,65635,THIS IS WHY YOU NEED\n\nA SHARPIE WITH YOU AT ...,[Black-and-white Fallacy/Dictatorship],https://www.facebook.com/photo/?fbid=402355213...
1,67927,GOOD NEWS!\n\nNAZANIN ZAGHARI-RATCLIFFE AND AN...,"[Loaded Language, Glittering generalities (Vir...",https://www.facebook.com/amnesty/photos/531198...
2,68031,PAING PHYO MIN IS FREE!,[],https://www.facebook.com/amnesty/photos/427419...
3,77490,Move your ships away!\n\noooook\n\nMove your s...,[],https://www.facebook.com/rightpatriots/photos/...
4,67641,"WHEN YOU'RE THE FBI, THEY LET YOU DO IT.",[Thought-terminating cliché],https://www.facebook.com/AddictingInfoOrg/phot...
...,...,...,...,...
495,78774,Level: Easy\n\nLevel: Hard\n\nLevel: For Mothe...,"[Slogans, Name calling/Labeling, Repetition]",https://www.facebook.com/themotherlandcalls/ph...
496,78299,Europeans:\n\nYou are trespassing,[],https://www.facebook.com/communism101/photos/5...
497,67096,\NONSENSE DETECTED! \nFIRE AT WILL!\\n\n\Time ...,[],https://www.facebook.com/photo/?fbid=181158225...
498,68464,ANNA AND VIRA:\nATTACKED FOR DEFENDING LGBTI A...,"[Causal Oversimplification, Loaded Language]",https://www.facebook.com/amnesty/photos/490311...


In [19]:
anno_subtask1_train.drop(columns=['link']).to_csv('./testdata/train/s1_train.csv')
anno_subtask1_val.drop(columns=['link']).to_csv('./testdata/val/s1_val.csv')

In [13]:
labels = list(set([x for xs in anno_subtask1_combined['labels'].to_list() for x in xs]))
labels

['Reductio ad hitlerum',
 "Misrepresentation of Someone's Position (Straw Man)",
 'Repetition',
 'Name calling/Labeling',
 'Presenting Irrelevant Data (Red Herring)',
 'Glittering generalities (Virtue)',
 'Bandwagon',
 'Loaded Language',
 'Smears',
 'Exaggeration/Minimisation',
 'Black-and-white Fallacy/Dictatorship',
 'Thought-terminating cliché',
 'Appeal to fear/prejudice',
 'Causal Oversimplification',
 'Obfuscation, Intentional vagueness, Confusion',
 'Flag-waving',
 'Slogans',
 'Whataboutism',
 'Doubt',
 'Appeal to authority']

# Transformers

Modified version of: https://github.com/kinit-sk/semeval2023-task3-persuasion-techniques

In [3]:
#reqs
import torch
import transformers
from torch.utils.data import Dataset
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer
import ast
import numpy as np
from torch.utils.data import DataLoader
from collections import Counter
from tqdm import tqdm
import pandas as pd
import os
from sklearn import metrics

## Transformer Classes

In [4]:
class BERTBase(torch.nn.Module):
    def __init__(self):
        super(BERTBase, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 19) # number of classes/techniques

    def forward(self, ids, mask, token_type_ids):
        _, output_1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output


class EnglishRobertaBase(torch.nn.Module):
    def __init__(self):
        super(EnglishRobertaBase, self).__init__()
        self.l1 = transformers.RobertaModel.from_pretrained('roberta-base', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 19)

    def forward(self, ids, mask, token_type_ids):
        _, output_1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output


class EnglishRobertaLarge(torch.nn.Module):
    def __init__(self):
        super(EnglishRobertaLarge, self).__init__()
        self.l1 = transformers.RobertaModel.from_pretrained('roberta-large', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(1024, 19)

    def forward(self, ids, mask, token_type_ids):
        _, output_1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output


class mBERTBase(torch.nn.Module):
    def __init__(self):
        super(mBERTBase, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-multilingual-cased', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 19)

    def forward(self, ids, mask, token_type_ids):
        _, output_1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output


class XLMRobertaBase(torch.nn.Module):
    def __init__(self):
        super(XLMRobertaBase, self).__init__()
        self.l1 = transformers.XLMRobertaModel.from_pretrained('xlm-roberta-base', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 19)

    def forward(self, ids, mask, token_type_ids):
        _, output_1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output


class XLMRobertaLarge(torch.nn.Module):
    def __init__(self):
        super(XLMRobertaLarge, self).__init__()
        self.l1 = transformers.XLMRobertaModel.from_pretrained('xlm-roberta-large', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(1024, 19)

    def forward(self, ids, mask, token_type_ids):
        _, output_1 = self.l1(ids, attention_mask=mask, token_type_ids=token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

## Data Loader

In [5]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.techniques_encoded
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }


def create_dataset_training(MODEL_NAME, PATHS_TO_TRAIN, PATHS_TO_VALIDATION, TRAINING_LANGUAGES):
    """
    Function to load and process the _input_data into suitable format. Includes setting up tokenizer, one hot encoding of
    techniques, concatenation of csv files if necessary, language filtering and loading _input_data into CustomDataset class.

    :param MODEL_NAME: name of the model as per Hugging face to use for tokenizer.
    :param PATHS_TO_TRAIN: List of paths to csv files we want to use for training.
    :param PATHS_TO_VALIDATION: List of paths to csv files we want to use for validation.
    :param TRAINING_LANGUAGES: List of languages we are using.
    :return: Tuple of training and testing set in Dataset format.
    """

    #Create tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, return_dict=False)
    MAX_LEN = tokenizer.model_max_length
    if MAX_LEN > 1024:
        print(f'hmm, seems the tokenizer is saying too much: {MAX_LEN}, setting down to 512.')
        MAX_LEN = 512
    print(f'max length of tokens for < {MODEL_NAME} > is: < {MAX_LEN} >')

    #Concatenate csv files
    df_train = pd.DataFrame()
    for train_path in PATHS_TO_TRAIN:
        df_train = pd.concat([df_train, pd.read_csv(train_path)], ignore_index=True)

    #One hot encoding of techniques
    df_train['labels'] = df_train['labels'].apply(ast.literal_eval)
    one_hot = MultiLabelBinarizer()
    df_train['techniques_encoded'] = one_hot.fit_transform(df_train['labels'])[:, 1:].tolist()

    #Load validation dataset and encode
    if PATHS_TO_VALIDATION:
        df_dev = pd.DataFrame()
        for dev_path in PATHS_TO_VALIDATION:
            df_dev = pd.concat([df_dev, pd.read_csv(dev_path)], ignore_index=True)

        df_dev['labels'] = df_dev['labels'].apply(ast.literal_eval)
        # use fitted one-hot encoder transform in case some techniques are present in training but not in dev
        df_dev['techniques_encoded'] = one_hot.transform(df_dev['labels'])[:, 1:].tolist()
    else:
        df_dev = pd.DataFrame(
            columns=[['id', 'text', 'labels', 'techniques_encoded']])

    #Language filtering - Not Needed yet
    # df_train = df_train[df_train['language'].isin(TRAINING_LANGUAGES)]
    # df_dev = df_dev[df_dev['language'].isin(TRAINING_LANGUAGES)]
    
    train_dataset = df_train
    test_dataset = df_dev

    print(f"TRAIN Dataset: {train_dataset.shape}")
    print(f"TEST Dataset: {test_dataset.shape}")

    training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
    testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

    return training_set, testing_set


def get_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name, return_dict=False)
    MAX_LEN = tokenizer.model_max_length
    if MAX_LEN > 1024:
        print(f'hmm, seems the tokenizer is saying too much: {MAX_LEN}, setting down to 512.')
        MAX_LEN = 512
    print(f'max length of tokens for < {MAX_LEN} > is: < {MAX_LEN} >')
    return tokenizer, MAX_LEN


def load_dataframe(args):
    test_df = pd.read_csv(args["EVALUATION_SET"])

    # language_to_texts = {lan: list(text) for lan, text in test_df.groupby('language')['text']}
    # for language, texts in language_to_texts.items():
    #     print(f'language {language} has {len(texts)} lines of data.')

    
    print('Encoding techniques into one-hot-encoded lists!\n')
        # Create sentence and label lists
    test_sentences = test_df.text.values

    test_df['labels'] = test_df['labels'].apply(ast.literal_eval)
    one_hot = MultiLabelBinarizer()
    test_df['techniques_encoded'] = one_hot.fit_transform(test_df['labels'])[:, 1:].tolist()
    test_labels = test_df.techniques_encoded.values
    test_labels = np.array([labels for labels in test_labels])

    mappings = one_hot.classes_[1:]    

    return test_df, mappings

def create_dataloader(args, test_df, language, language_counts):
    # print(
    #     f'testing on {language_counts[language]} data points (around {language_counts[language] / args["BATCH_SIZE"]} iterations)')
    filtered_test_df = test_df.copy()

    # not required until we use multilingual data
    # filtered_test_df = filtered_test_df[filtered_test_df['language'] == language]
    # filtered_test_df = filtered_test_df.reset_index()
    
    tokenizer, MAX_LEN = get_tokenizer(args['MODEL_NAME'])

    testing_set = CustomDataset(
        test_df,
        tokenizer,
        MAX_LEN
    )

    testing_loader = DataLoader(
        testing_set,
        batch_size=args['BATCH_SIZE'],
        shuffle=args['SHUFFLE'],
        num_workers=args['NUM_WORKERS']
    )

    return testing_loader, filtered_test_df

## Training

In [6]:
def run_training(args):
    """
    Function creates a dataset based on given arguments. Afterwards it trains and validates a model.

    :param args: dictionary of arguments needed to create the model
    """
    training_set, testing_set = create_dataset_training(MODEL_NAME=args['MODEL_NAME'], PATHS_TO_TRAIN=args['PATHS_TO_TRAIN'], PATHS_TO_VALIDATION=args['PATHS_TO_VALIDATION'], TRAINING_LANGUAGES=args['TRAINING_LANGUAGES'])

    training_loader = DataLoader(
        training_set,
        batch_size=args['TRAIN_BATCH_SIZE'],
        shuffle=args['TRAIN_SHUFFLE'],
        num_workers=args['TRAIN_NUM_WORKERS']
    )

    validation_loader = DataLoader(
        testing_set,
        batch_size=args['VALIDATION_BATCH_SIZE'],
        shuffle=args['VALIDATION_SHUFFLE'],
        num_workers=args['VALIDATION_NUM_WORKERS']
    )

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    lm = LanguageModel(training_loader=training_loader, validation_loader=validation_loader, device=device, args=args)
    return lm.train_over_epochs()


class LanguageModel:
    """
    Class to hold the model as well as train and validate it.
    """

    def __init__(self, training_loader, validation_loader, device, args):

        self.args = args
        self.device = device
        self.validation_loader = validation_loader
        self.training_loader = training_loader
        if args['MODEL_NAME'] == 'roberta-large':
            self.model = EnglishRobertaLarge()
        elif args['MODEL_NAME'] == 'roberta-base':
            self.model = EnglishRobertaBase()
        elif args['MODEL_NAME'] == 'bert-base-uncased':
            self.model = BERTBase()
        elif args['MODEL_NAME'] == 'bert-base-multilingual-cased':
            self.model = mBERTBase()
        elif args['MODEL_NAME'] == 'xlm-roberta-base':
            self.model = XLMRobertaBase()
        elif args['MODEL_NAME'] == 'xlm-roberta-large':
            self.model = XLMRobertaLarge()

        self.model.to(self.device)
        self.optimizer = torch.optim.Adam(params=self.model.parameters(), lr=self.args['LEARNING_RATE'])
        if args['FREEZE']:
            counter = 0
            no_layers = len([param for param in self.model.parameters()])
            for param in self.model.parameters():
                counter += 1
                param.requires_grad = False
                if counter == (no_layers//5) * 4:
                    break


    def save_model_and_weights(self, epoch):
        print(f'\n\n --- Currently saving model! --- \n')
        model_path = f'{self.args["MODELS_DIR"]}{self.args["MODEL_NAME_PREFIX"]}_model_{epoch}epoch_batch{self.args["TRAIN_BATCH_SIZE"]}_lr{self.args["LEARNING_RATE"]}.pth'
        model_weights_path = f'{self.args["MODELS_DIR"]}{self.args["MODEL_NAME_PREFIX"]}_model_weights_{epoch}epoch_batch{self.args["TRAIN_BATCH_SIZE"]}_lr{self.args["LEARNING_RATE"]}.pth'
        torch.save(self.model.state_dict(), model_weights_path)
        torch.save(self.model, model_path)
        return model_path

    def train(self, epoch):
        print(f'\nNow training on Epoch {epoch}.')
        print(
            f'Testing on {len(self.training_loader) * self.training_loader.batch_size - 1} data points (around {len(self.training_loader)} iterations)')

        self.args['CURRENT_TRAIN_LOSS'] = 0
        self.model.train()
        for itr, data in tqdm(enumerate(self.training_loader, 0)):
            ids = data['ids'].to(self.device, dtype=torch.long)
            mask = data['mask'].to(self.device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(self.device, dtype=torch.long)
            targets = data['targets'].to(self.device, dtype=torch.float)

            outputs = self.model(ids, mask, token_type_ids)

            loss = torch.nn.BCEWithLogitsLoss()(outputs, targets)
            if itr % 200 == 0:
                print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

            self.optimizer.zero_grad()

            loss.backward()
            self.optimizer.step()

            self.args['CURRENT_TRAIN_LOSS'] += loss

        print(f'Accumulated Training Loss after Epoch {epoch} is: {self.args["CURRENT_TRAIN_LOSS"]}')

    def validate(self, epoch):
        print(f'\nNow validating on Epoch {epoch}.')
        print(
            f'Validating on {len(self.validation_loader) * self.validation_loader.batch_size} data points (around {len(self.validation_loader)} iterations)')
        self.model.eval()
        self.args['CURRENT_VALIDATION_LOSS'] = 0
        with torch.no_grad():
            for val_itr, data in tqdm(enumerate(self.validation_loader)):
                ids = data['ids'].to(self.device, dtype=torch.long)
                mask = data['mask'].to(self.device, dtype=torch.long)
                token_type_ids = data['token_type_ids'].to(self.device, dtype=torch.long)
                targets = data['targets'].to(self.device, dtype=torch.float)

                outputs = self.model(ids, mask, token_type_ids)
                loss = torch.nn.BCEWithLogitsLoss()(outputs, targets)

                if val_itr % 200 == 0:
                    print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

                # accumulate the loss across all batches
                self.args['CURRENT_VALIDATION_LOSS'] += loss

                # clear cache
                torch.cuda.empty_cache()

        print(f'Accumulated Validation Loss after Epoch {epoch} is: {self.args["CURRENT_VALIDATION_LOSS"]}')

        # Early stopping if necessary
        if self.args['CURRENT_VALIDATION_LOSS'] < self.args['BEST_LOSS']:
            self.args['BEST_LOSS'] = self.args['CURRENT_VALIDATION_LOSS']
            self.save_model_and_weights(epoch)

            return True
        else:
            # stop training
            print('Validation loss increased! Stopping training...')
            print(f'Last validation loss was: {self.args["BEST_LOSS"]}')
            print(f'Current validation loss is: {self.args["CURRENT_VALIDATION_LOSS"]}')

            return False

    def train_over_epochs(self):
        for epoch in range(self.args['EPOCHS']):
            self.train(epoch)
            if self.args['PATHS_TO_VALIDATION'] and not self.validate(epoch):
                break  # the last epoch was NOT counted due to early stopping
            else:
                # store latest version
                path = self.save_model_and_weights(epoch)
        print("Done.")
        return path

# Training Different Models (should be a loop but is not a loop)

In [42]:
args_train = {
    # general
    'MODEL_NAME_PREFIX': '',                                # so that model names don't get overwritten, write a UNIQUE name prefix to distinguish this current model
    'EPOCHS': 5,                                            # Make sure to set the EXACT amount of epochs for non-validating runs. Typically BERT-like models won't need to go over 4 epochs
    'LEARNING_RATE': 1e-05,
    'MODEL_NAME': 'xlm-roberta-large',                      # model name as per hugging face library, currently supported model names are: roberta-large, roberta-base, bert-base-uncased, bert-base-multilingual-cased, xlm-roberta-base and xlm-roberta-large
    'DROPOUT_RATE': 0.3,
    'MODELS_DIR': './unimodal-baselines',               # name of subdirectory within 'models' folder where the trained models will be saved, e.g., 'distilbert_base_threshold_0_4'
    'PATHS_TO_TRAIN': ['./testdata/s1_train.csv'],    # useful when training on more sets (e.g. train and dev), list of paths to the csv files
    'PATHS_TO_VALIDATION': ['./testdata/s1_val.csv'],                              # leave empty if no validation required

    # for training
    'TRAIN_BATCH_SIZE': 8,
    'TRAIN_SHUFFLE': True,
    'TRAIN_NUM_WORKERS': 0,
    'CURRENT_EPOCH': 0,
    'CURRENT_TRAIN_LOSS': 0,
    'TRAINING_LANGUAGES': ['en'], # full list is ['en', 'fr', 'ge', 'it', 'po', 'ru']
    'FREEZE': False,                                        # If True first 80% of the layers will be frozen during training
                                                            # unfreeze for PTC fine-tuning?

    # for validation
    'VALIDATION_BATCH_SIZE': 8,
    'VALIDATION_SHUFFLE': False,
    'VALIDATION_NUM_WORKERS': 0,
    'BEST_LOSS': float('inf'),
    'CURRENT_VALIDATION_LOSS': 0
}

model_file = run_training(args_train)

max length of tokens for < xlm-roberta-large > is: < 512 >
TRAIN Dataset: (7000, 5)
TEST Dataset: (500, 5)

Now training on Epoch 0.
Testing on 6999 data points (around 875 iterations)


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\users\goat\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Training Loss:  0.6796550750732422


200it [38:37, 11.59s/it]

Epoch: 0, Training Loss:  0.22325746715068817


...

Accumulated Training Loss after Epoch 4 is: 113.25543975830078

Now validating on Epoch 4.
Validating on 504 data points (around 63 iterations)


0it [00:00, ?it/s]

Epoch: 4, Training Loss:  0.20653827488422394


63it [04:00,  3.82s/it]


Accumulated Validation Loss after Epoch 4 is: 12.515527725219727
Validation loss increased! Stopping training...
Last validation loss was: 12.381818771362305
Current validation loss is: 12.515527725219727
Done.


In [44]:
args_train = {
    # general
    'MODEL_NAME_PREFIX': 'bert-base-multilingual-cased',                                # so that model names don't get overwritten, write a UNIQUE name prefix to distinguish this current model
    'EPOCHS': 5,                                            # Make sure to set the EXACT amount of epochs for non-validating runs. Typically BERT-like models won't need to go over 4 epochs
    'LEARNING_RATE': 1e-05,
    'MODEL_NAME': 'bert-base-multilingual-cased',                      # model name as per hugging face library, currently supported model names are: roberta-large, roberta-base, bert-base-uncased, bert-base-multilingual-cased, xlm-roberta-base and xlm-roberta-large
    'DROPOUT_RATE': 0.3,
    'MODELS_DIR': './unimodal-baselines/',               # name of subdirectory within 'models' folder where the trained models will be saved, e.g., 'distilbert_base_threshold_0_4'
    'PATHS_TO_TRAIN': ['./testdata/s1_train.csv'],    # useful when training on more sets (e.g. train and dev), list of paths to the csv files
    'PATHS_TO_VALIDATION': ['./testdata/s1_val.csv'],                              # leave empty if no validation required

    # for training
    'TRAIN_BATCH_SIZE': 8,
    'TRAIN_SHUFFLE': True,
    'TRAIN_NUM_WORKERS': 0,
    'CURRENT_EPOCH': 0,
    'CURRENT_TRAIN_LOSS': 0,
    'TRAINING_LANGUAGES': ['en'], # full list is ['en', 'fr', 'ge', 'it', 'po', 'ru']
    'FREEZE': False,                                        # If True first 80% of the layers will be frozen during training

    # for validation
    'VALIDATION_BATCH_SIZE': 8,
    'VALIDATION_SHUFFLE': False,
    'VALIDATION_NUM_WORKERS': 0,
    'BEST_LOSS': float('inf'),
    'CURRENT_VALIDATION_LOSS': 0
}

model_file = run_training(args_train)

max length of tokens for < bert-base-multilingual-cased > is: < 512 >
TRAIN Dataset: (7000, 5)
TEST Dataset: (500, 5)

Now training on Epoch 0.
Testing on 6999 data points (around 875 iterations)


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\users\goat\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Training Loss:  0.6773495674133301


200it [08:57,  2.67s/it]

Epoch: 0, Training Loss:  0.28486016392707825


...

Accumulated Training Loss after Epoch 4 is: 159.1197052001953

Now validating on Epoch 4.
Validating on 504 data points (around 63 iterations)


1it [00:02,  2.02s/it]

Epoch: 4, Training Loss:  0.23882943391799927


63it [00:16,  3.83it/s]

Accumulated Validation Loss after Epoch 4 is: 13.922234535217285
Validation loss increased! Stopping training...
Last validation loss was: 13.681135177612305
Current validation loss is: 13.922234535217285
Done.


In [45]:
args_train = {
    # general
    'MODEL_NAME_PREFIX': 'roberta-large',                                # so that model names don't get overwritten, write a UNIQUE name prefix to distinguish this current model
    'EPOCHS': 5,                                            # Make sure to set the EXACT amount of epochs for non-validating runs. Typically BERT-like models won't need to go over 4 epochs
    'LEARNING_RATE': 1e-05,
    'MODEL_NAME': 'roberta-large',                      # model name as per hugging face library, currently supported model names are: roberta-large, roberta-base, bert-base-uncased, bert-base-multilingual-cased, xlm-roberta-base and xlm-roberta-large
    'DROPOUT_RATE': 0.3,
    'MODELS_DIR': './unimodal-baselines/',               # name of subdirectory within 'models' folder where the trained models will be saved, e.g., 'distilbert_base_threshold_0_4'
    'PATHS_TO_TRAIN': ['./testdata/s1_train.csv'],    # useful when training on more sets (e.g. train and dev), list of paths to the csv files
    'PATHS_TO_VALIDATION': ['./testdata/s1_val.csv'],                              # leave empty if no validation required

    # for training
    'TRAIN_BATCH_SIZE': 8,
    'TRAIN_SHUFFLE': True,
    'TRAIN_NUM_WORKERS': 0,
    'CURRENT_EPOCH': 0,
    'CURRENT_TRAIN_LOSS': 0,
    'TRAINING_LANGUAGES': ['en'], # full list is ['en', 'fr', 'ge', 'it', 'po', 'ru']
    'FREEZE': False,                                        # If True first 80% of the layers will be frozen during training

    # for validation
    'VALIDATION_BATCH_SIZE': 8,
    'VALIDATION_SHUFFLE': False,
    'VALIDATION_NUM_WORKERS': 0,
    'BEST_LOSS': float('inf'),
    'CURRENT_VALIDATION_LOSS': 0
}

model_file = run_training(args_train)

max length of tokens for < roberta-large > is: < 512 >
TRAIN Dataset: (7000, 5)
TEST Dataset: (500, 5)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Now training on Epoch 0.
Testing on 6999 data points (around 875 iterations)


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\users\goat\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Training Loss:  0.7196247577667236


...

Accumulated Training Loss after Epoch 4 is: 103.2136001586914

Now validating on Epoch 4.
Validating on 504 data points (around 63 iterations)


0it [00:00, ?it/s]

Epoch: 4, Training Loss:  0.18887023627758026


63it [00:10,  6.09it/s]


Accumulated Validation Loss after Epoch 4 is: 12.16552734375
Validation loss increased! Stopping training...
Last validation loss was: 12.021674156188965
Current validation loss is: 12.16552734375
Done.


In [46]:
args_train = {
    # general
    'MODEL_NAME_PREFIX': 'roberta-base',                                # so that model names don't get overwritten, write a UNIQUE name prefix to distinguish this current model
    'EPOCHS': 5,                                            # Make sure to set the EXACT amount of epochs for non-validating runs. Typically BERT-like models won't need to go over 4 epochs
    'LEARNING_RATE': 1e-05,
    'MODEL_NAME': 'roberta-base',                      # model name as per hugging face library, currently supported model names are: roberta-large, roberta-base, bert-base-uncased, bert-base-multilingual-cased, xlm-roberta-base and xlm-roberta-large
    'DROPOUT_RATE': 0.3,
    'MODELS_DIR': './unimodal-baselines/',               # name of subdirectory within 'models' folder where the trained models will be saved, e.g., 'distilbert_base_threshold_0_4'
    'PATHS_TO_TRAIN': ['./testdata/s1_train.csv'],    # useful when training on more sets (e.g. train and dev), list of paths to the csv files
    'PATHS_TO_VALIDATION': ['./testdata/s1_val.csv'],                              # leave empty if no validation required

    # for training
    'TRAIN_BATCH_SIZE': 8,
    'TRAIN_SHUFFLE': True,
    'TRAIN_NUM_WORKERS': 0,
    'CURRENT_EPOCH': 0,
    'CURRENT_TRAIN_LOSS': 0,
    'TRAINING_LANGUAGES': ['en'], # full list is ['en', 'fr', 'ge', 'it', 'po', 'ru']
    'FREEZE': False,                                        # If True first 80% of the layers will be frozen during training

    # for validation
    'VALIDATION_BATCH_SIZE': 8,
    'VALIDATION_SHUFFLE': False,
    'VALIDATION_NUM_WORKERS': 0,
    'BEST_LOSS': float('inf'),
    'CURRENT_VALIDATION_LOSS': 0
}

model_file = run_training(args_train)

max length of tokens for < roberta-base > is: < 512 >
TRAIN Dataset: (7000, 5)
TEST Dataset: (500, 5)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Now training on Epoch 0.
Testing on 6999 data points (around 875 iterations)


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch: 0, Training Loss:  0.7152462005615234


...

Accumulated Training Loss after Epoch 4 is: 136.10641479492188

Now validating on Epoch 4.
Validating on 504 data points (around 63 iterations)


3it [00:00, 21.30it/s]

Epoch: 4, Training Loss:  0.23383788764476776


63it [00:03, 19.12it/s]

Accumulated Validation Loss after Epoch 4 is: 12.972590446472168
Validation loss increased! Stopping training...
Last validation loss was: 12.661491394042969
Current validation loss is: 12.972590446472168
Done.


In [10]:
args_train = {
    # general
    'MODEL_NAME_PREFIX': 'bert-base-uncased',                                # so that model names don't get overwritten, write a UNIQUE name prefix to distinguish this current model
    'EPOCHS': 1,                                            # Make sure to set the EXACT amount of epochs for non-validating runs. Typically BERT-like models won't need to go over 4 epochs
    'LEARNING_RATE': 1e-05,
    'MODEL_NAME': 'bert-base-uncased',                      # model name as per hugging face library, currently supported model names are: roberta-large, roberta-base, bert-base-uncased, bert-base-multilingual-cased, xlm-roberta-base and xlm-roberta-large
    'DROPOUT_RATE': 0.3,
    'MODELS_DIR': './unimodal-baselines/',               # name of subdirectory within 'models' folder where the trained models will be saved, e.g., 'distilbert_base_threshold_0_4'
    'PATHS_TO_TRAIN': ['./testdata/s1_train.csv'],    # useful when training on more sets (e.g. train and dev), list of paths to the csv files
    'PATHS_TO_VALIDATION': ['./testdata/s1_val.csv'],                              # leave empty if no validation required

    # for training
    'TRAIN_BATCH_SIZE': 8,
    'TRAIN_SHUFFLE': True,
    'TRAIN_NUM_WORKERS': 0,
    'CURRENT_EPOCH': 0,
    'CURRENT_TRAIN_LOSS': 0,
    'TRAINING_LANGUAGES': ['en'], # full list is ['en', 'fr', 'ge', 'it', 'po', 'ru']
    'FREEZE': False,                                        # If True first 80% of the layers will be frozen during training

    # for validation
    'VALIDATION_BATCH_SIZE': 8,
    'VALIDATION_SHUFFLE': False,
    'VALIDATION_NUM_WORKERS': 0,
    'BEST_LOSS': float('inf'),
    'CURRENT_VALIDATION_LOSS': 0
}

model_file = run_training(args_train)

max length of tokens for < bert-base-uncased > is: < 512 >
TRAIN Dataset: (7000, 5)
TEST Dataset: (500, 5)

Now training on Epoch 0.
Testing on 6999 data points (around 875 iterations)


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\Goat\AppData\Roaming\Python\Python38\site-packages\transformers\tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Training Loss:  0.6837515830993652


200it [08:00,  2.44s/it]

Epoch: 0, Training Loss:  0.2346428781747818


206it [08:17,  2.42s/it]


KeyboardInterrupt: 

In [48]:
args_train = {
    # general
    'MODEL_NAME_PREFIX': 'bert-base-multilingual-cased',                                # so that model names don't get overwritten, write a UNIQUE name prefix to distinguish this current model
    'EPOCHS': 5,                                            # Make sure to set the EXACT amount of epochs for non-validating runs. Typically BERT-like models won't need to go over 4 epochs
    'LEARNING_RATE': 1e-05,
    'MODEL_NAME': 'bert-base-multilingual-cased',                      # model name as per hugging face library, currently supported model names are: roberta-large, roberta-base, bert-base-uncased, bert-base-multilingual-cased, xlm-roberta-base and xlm-roberta-large
    'DROPOUT_RATE': 0.3,
    'MODELS_DIR': './unimodal-baselines/',               # name of subdirectory within 'models' folder where the trained models will be saved, e.g., 'distilbert_base_threshold_0_4'
    'PATHS_TO_TRAIN': ['./testdata/s1_train.csv'],    # useful when training on more sets (e.g. train and dev), list of paths to the csv files
    'PATHS_TO_VALIDATION': ['./testdata/s1_val.csv'],                              # leave empty if no validation required

    # for training
    'TRAIN_BATCH_SIZE': 8,
    'TRAIN_SHUFFLE': True,
    'TRAIN_NUM_WORKERS': 0,
    'CURRENT_EPOCH': 0,
    'CURRENT_TRAIN_LOSS': 0,
    'TRAINING_LANGUAGES': ['en'], # full list is ['en', 'fr', 'ge', 'it', 'po', 'ru']
    'FREEZE': False,                                        # If True first 80% of the layers will be frozen during training

    # for validation
    'VALIDATION_BATCH_SIZE': 8,
    'VALIDATION_SHUFFLE': False,
    'VALIDATION_NUM_WORKERS': 0,
    'BEST_LOSS': float('inf'),
    'CURRENT_VALIDATION_LOSS': 0
}

model_file = run_training(args_train)

max length of tokens for < bert-base-multilingual-cased > is: < 512 >
TRAIN Dataset: (7000, 5)
TEST Dataset: (500, 5)

Now training on Epoch 0.
Testing on 6999 data points (around 875 iterations)


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch: 0, Training Loss:  0.6666839718818665

c:\users\goat\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
1it [00:00,  2.95it/s]

201it [00:33,  5.94it/s]

...

Accumulated Training Loss after Epoch 4 is: 153.6635284423828

Now validating on Epoch 4.
Validating on 504 data points (around 63 iterations)


3it [00:00, 20.54it/s]

Epoch: 4, Training Loss:  0.2382974624633789


63it [00:03, 18.65it/s]


Accumulated Validation Loss after Epoch 4 is: 13.570302963256836
Validation loss increased! Stopping training...
Last validation loss was: 13.5440092086792
Current validation loss is: 13.570302963256836
Done.


In [49]:
args_train = {
    # general
    'MODEL_NAME_PREFIX': 'xlm-roberta-base',                                # so that model names don't get overwritten, write a UNIQUE name prefix to distinguish this current model
    'EPOCHS': 5,                                            # Make sure to set the EXACT amount of epochs for non-validating runs. Typically BERT-like models won't need to go over 4 epochs
    'LEARNING_RATE': 1e-05,
    'MODEL_NAME': 'xlm-roberta-base',                      # model name as per hugging face library, currently supported model names are: roberta-large, roberta-base, bert-base-uncased, bert-base-multilingual-cased, xlm-roberta-base and xlm-roberta-large
    'DROPOUT_RATE': 0.3,
    'MODELS_DIR': './unimodal-baselines/',               # name of subdirectory within 'models' folder where the trained models will be saved, e.g., 'distilbert_base_threshold_0_4'
    'PATHS_TO_TRAIN': ['./testdata/s1_train.csv'],    # useful when training on more sets (e.g. train and dev), list of paths to the csv files
    'PATHS_TO_VALIDATION': ['./testdata/s1_val.csv'],                              # leave empty if no validation required

    # for training
    'TRAIN_BATCH_SIZE': 8,
    'TRAIN_SHUFFLE': True,
    'TRAIN_NUM_WORKERS': 0,
    'CURRENT_EPOCH': 0,
    'CURRENT_TRAIN_LOSS': 0,
    'TRAINING_LANGUAGES': ['en'], # full list is ['en', 'fr', 'ge', 'it', 'po', 'ru']
    'FREEZE': False,                                        # If True first 80% of the layers will be frozen during training

    # for validation
    'VALIDATION_BATCH_SIZE': 8,
    'VALIDATION_SHUFFLE': False,
    'VALIDATION_NUM_WORKERS': 0,
    'BEST_LOSS': float('inf'),
    'CURRENT_VALIDATION_LOSS': 0
}

model_file = run_training(args_train)

max length of tokens for < xlm-roberta-base > is: < 512 >
TRAIN Dataset: (7000, 5)
TEST Dataset: (500, 5)



Now training on Epoch 0.
Testing on 6999 data points (around 875 iterations)


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\users\goat\appdata\local\programs\python\python38\lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
1it [00:00,  5.10it/s]

Epoch: 0, Training Loss:  0.7227253913879395


201it [00:35,  5.60it/s]

...

Accumulated Training Loss after Epoch 4 is: 180.2266082763672

Now validating on Epoch 4.
Validating on 504 data points (around 63 iterations)


3it [00:00, 20.70it/s]

Epoch: 4, Training Loss:  0.23572345077991486


63it [00:03, 19.12it/s]


Accumulated Validation Loss after Epoch 4 is: 13.673762321472168


 --- Currently saving model! --- 



 --- Currently saving model! --- 

Done.


# Evaluation

In [35]:
def test(model, testing_loader, only_test):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)

            # if there are no targets available
            if not only_test:
                targets = data['targets'].to(device, dtype=torch.float)
                fin_targets.extend(targets.cpu().detach().numpy().tolist())

            outputs = model(ids, mask, token_type_ids)
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets


def run_tests(model, testing_loader, evaluation_threshold, only_test, lang, results_list):
    outputs, targets = test(model, testing_loader, only_test)
    outputs = np.array(outputs) >= evaluation_threshold

    if targets:
        accuracy = metrics.accuracy_score(targets, outputs)
        f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
        f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
        print(f"Accuracy Score = {accuracy}")
        print(f"F1 Score (Micro) = {f1_score_micro}")
        print(f"F1 Score (Macro) = {f1_score_macro}")
        results_list.append([lang, accuracy, f1_score_micro, f1_score_macro])
        print(results_list)

    return outputs, results_list

def run_eval(args):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = torch.load(args["MODEL_FILE_NAME"])
    model.to(device)

    test_df, mappings = load_dataframe(args)
    language_counts = 1 # hard code to one since just testing on English
    language = args['LANGUAGES']

    results_list = []
    
    testing_loader, filtered_test_df = create_dataloader(args, test_df, args['LANGUAGES'], language_counts)

    outputs, results_list = run_tests(model, testing_loader, args['EVALUATION_THRESHOLD'], args['ONLY_TEST'], language, results_list)

    #path = f'results/{args["RESULT_DIR_NAME"]}/{language}.txt'

    df_results = pd.DataFrame(results_list, columns=['language', 'accuracy', 'F1_micro', 'F1_macro'])
    df_results['model'] = args['MODEL_TITLE']
    return df_results


In [37]:
#models names: roberta-large, roberta-base, bert-base-uncased, bert-base-multilingual-cased, xlm-roberta-base and xlm-roberta-large

args_eval = {
    'MODEL_FILE_NAME': path, # path to saved model
    'RESULT_DIR_NAME': '',                                  # name of subdirectory within 'results' folder where to store the txt result files (for submission), e.g., 'distilbert_base_threshold_0_4'
    'EVALUATION_THRESHOLD': 0.5,                            # select the confidence threshold for when to assign a label. Default is 0.5
    'MODEL_NAME': 'bert-base-uncased',
    'MODEL_TITLE': 'title',
    'ONLY_TEST': False,                                      # set to `True` ONLY IF running on test set (not dev set or anything else)
    'EVALUATION_SET': './testdata/s1_val.csv',       # path to csv with evaluation set
    'LANGUAGES': ['en'],    # select which languages to evaluate, packaged in a list. # e.g. ['en', 'fr', 'ge', 'it', 'po', 'ru', 'ka', 'gr', 'es'],

    # test params
    'BATCH_SIZE': 32,
    'SHUFFLE': False,  
    'NUM_WORKERS': 0
}

#run_eval(args_eval)

In [39]:
path = r'change this to where the models are saved'
models_path = os.listdir(r'change this to where the models are saved')

models = [i for i in models_path if 'weights' not in i]
print(models)

['bert_base.pth', 'mbert.pth', 'roberta_base.pth', 'roberta_large.pth', 'xlm_roberta_base.pth', 'xlm_roberta_large.pth']


In [42]:
results_df = pd.DataFrame()

for i in models:
    args_eval['MODEL_FILE_NAME'] = os.path.join(path, i)
    args_eval['MODEL_TITLE'] = i.split('.')[0]
    results = run_eval(args_eval)
    results_df = pd.concat([results_df, results])
    results_df

Encoding techniques into one-hot-encoded lists!

max length of tokens for < 512 > is: < 512 >


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\Goat\AppData\Roaming\Python\Python38\site-packages\transformers\tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
16it [00:03,  4.64it/s]


Accuracy Score = 0.214
F1 Score (Micro) = 0.291015625
F1 Score (Macro) = 0.10118576656955787
[[['en'], 0.214, 0.291015625, 0.10118576656955787]]
Encoding techniques into one-hot-encoded lists!

max length of tokens for < 512 > is: < 512 >


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\Goat\AppData\Roaming\Python\Python38\site-packages\transformers\tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
16it [00:03,  4.68it/s]


Accuracy Score = 0.21
F1 Score (Micro) = 0.1218961625282167
F1 Score (Macro) = 0.028919578560726886
[[['en'], 0.21, 0.1218961625282167, 0.028919578560726886]]
Encoding techniques into one-hot-encoded lists!

max length of tokens for < 512 > is: < 512 >


...

Accuracy Score = 0.116
F1 Score (Micro) = 0.13050847457627118
F1 Score (Macro) = 0.033793371329977485
[[['en'], 0.116, 0.13050847457627118, 0.033793371329977485]]
Encoding techniques into one-hot-encoded lists!

max length of tokens for < 512 > is: < 512 >


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\Goat\AppData\Roaming\Python\Python38\site-packages\transformers\tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
16it [00:03,  4.92it/s]


Accuracy Score = 0.2
F1 Score (Micro) = 0.0731995277449823
F1 Score (Macro) = 0.026169902367859946
[[['en'], 0.2, 0.0731995277449823, 0.026169902367859946]]
Encoding techniques into one-hot-encoded lists!

max length of tokens for < 512 > is: < 512 >


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\Goat\AppData\Roaming\Python\Python38\site-packages\transformers\tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
16it [00:03,  4.89it/s]

Accuracy Score = 0.188
F1 Score (Micro) = 0.010217113665389528
F1 Score (Macro) = 0.0029651593773165306
[[['en'], 0.188, 0.010217113665389528, 0.0029651593773165306]]


In [44]:
results_df

,language,accuracy,F1_micro,F1_macro,model
0,[en],0.214,0.291016,0.101186,bert_base
0,[en],0.210,0.121896,0.028920,mbert
0,[en],0.136,0.158940,0.038990,roberta_base
0,[en],0.116,0.130508,0.033793,roberta_large
0,[en],0.200,0.073200,0.026170,xlm_roberta_base
0,[en],0.188,0.010217,0.002965,xlm_roberta_large
